In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
class LSTM_Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTM_Autoencoder, self).__init__()
        
        # Encoder (LSTM)
        self.encoder = nn.Sequential(nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True),
                                     nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True))
        
        # Decoder (LSTM)
        self.decoder = nn.Sequential(nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True),
                                     nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True),
                                     nn.Dropout(dropout_rate))
        
        # Output layer (to map the hidden state back to input size)
        self.output_layer = nn.Linear(hidden_size, input_size)
        
    def forward(self, x):
        # Encoding phase
        _, (hn, _) = self.encoder(x)
        
        # Repeat hidden state for each time step (for decoding)
        h_n = hn[-1].unsqueeze(1).repeat(1, x.size(1), 1)
        
        # Decoding phase
        decoded, _ = self.decoder(h_n)
        
        # Output prediction
        output = self.output_layer(decoded)
        
        return output


In [ ]:
# ハイパーパラメータ
input_size = 128  # 入力次元
hidden_size = 128  # LSTMの隠れ層のユニット数
num_layers = 2  # LSTMの層数
dropout_rate = 0.2  # ドロップアウトの割合
learning_rate = 0.0025  # 学習率
epochs = 50  # 最大エポック数
batch_size = 64  # バッチサイズ


In [ ]:
# ダミーデータの作成
def generate_synthetic_data(num_samples, seq_length, input_size):
    data = np.random.rand(num_samples, seq_length, input_size).astype(np.float32)
    return torch.tensor(data)

# データの準備
num_samples = 1000
seq_length = 10  # シーケンスの長さ
data = generate_synthetic_data(num_samples, seq_length, input_size)

# データセットの作成
dataset = TensorDataset(data, data)  # オートエンコーダでは、入力とターゲットが同じ
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# モデルのインスタンス化
model = LSTM_Autoencoder(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)

# モデルのデバイスへの移動
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 最適化手法の定義
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# 損失関数の定義（MSE損失を使用）
criterion = nn.MSELoss()

# 学習ループ
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # 順伝播
        outputs = model(inputs)
        
        # 損失の計算
        loss = criterion(outputs, targets)
        
        # 勾配の初期化と逆伝播
        optimizer.zero_grad()
        loss.backward()
        
        # パラメータの更新
        optimizer.step()
        
        # バッチごとの損失を記録
        running_loss += loss.item()
        
    # 1エポックごとに損失を表示
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(data_loader):.4f}")


In [ ]:
# 評価モードに切り替え
model.eval()

# 予測の実行
with torch.no_grad():
    test_data = generate_synthetic_data(100, seq_length, input_size)  # テストデータの生成
    test_data = torch.tensor(test_data).to(device)
    outputs = model(test_data)
    
    # 出力の表示（ここでは一部の出力を表示）
    print("Predicted output:", outputs[0].cpu().numpy())
